In [1]:

import tensorflow as tf
 
x1 = tf.placeholder(dtype=tf.float32, shape=[], name = 'x1')
 
x2 = tf.placeholder(dtype=tf.float32, shape=[], name = 'x2')
 
w = tf.Variable(tf.constant(2.), name = 'w')
 
ytmp = tf.multiply(w, x1, name = 'ytmp')
 
y = tf.add(ytmp, x2, name = 'y')
 
sess = tf.Session()
 
sess.run(tf.global_variables_initializer())
 
print (sess.run(y, feed_dict={x1: 1., x2: 2.}))
 
saver = tf.train.Saver()
 
saver.save(sess, 'model/test')


4.0


'model/test'

In [2]:
import tensorflow as tf
 
sess = tf.Session()
 
# 导入运算图
saver = tf.train.import_meta_graph('model/test.meta')
 
#加载相应参数
saver.restore(sess, tf.train.latest_checkpoint('model/'))
 
graph = tf.get_default_graph()
 
x1 = graph.get_tensor_by_name('x1:0')
 
x2 = graph.get_tensor_by_name('x2:0')
 
y = graph.get_tensor_by_name('y:0')
 
# restore之后不需要执行variable初始化
#sess.run(tf.global_variables_initializer())
print (sess.run(graph.get_tensor_by_name('w:0')))
print (sess.run(y, feed_dict={x1: 1., x2: 2.}))

INFO:tensorflow:Restoring parameters from model/test
2.0
4.0


In [1]:
sess

NameError: name 'sess' is not defined

In [12]:
def predict(sess, inputs):
    preds = sess.run(y, feed_dict={x1: inputs, x2: 2.0}) 
    # `X` is used, it must be defined in the model with that name explicitly!
    return [str(p) for p in preds]

In [12]:
def predict(sess, inputs):
    preds = sess.run(sy ,feed_dict={"x:0": inputs}) 
    # `X` is used, it must be defined in the model with that name explicitly!
    return [str(p) for p in preds]

In [15]:
predict(sess,[])

ValueError: Cannot feed value of shape (0,) for Tensor 'x1:0', which has shape '()'

Cannot feed value of shape (0,) for Tensor 'x1:0', which has shape '()'  两个位置代表哪里 

tensor是个 ndarrange？ 我的输入一定要是ndarrange？

In [18]:

from clipper_admin import ClipperConnection, DockerContainerManager
from clipper_admin.deployers.tensorflow import deploy_tensorflow_model
clipper_conn = ClipperConnection(DockerContainerManager())

In [19]:
clipper_conn.start_clipper()

20-03-13:06:05:32 INFO     [docker_container_manager.py:184] [default-cluster] Starting managed Redis instance in Docker
20-03-13:06:05:36 INFO     [docker_container_manager.py:276] [default-cluster] Metric Configuration Saved at /tmp/tmpm4x21hg6.yml
20-03-13:06:05:37 INFO     [clipper_admin.py:162] [default-cluster] Clipper is running


In [20]:
clipper_conn.connect()

20-03-13:06:05:40 INFO     [clipper_admin.py:172] [default-cluster] Successfully connected to Clipper cluster at localhost:1337


In [21]:

# Add an application with a name and an input type
clipper_conn.register_application(
    name="tf-app2", input_type="doubles", default_output="-1.0", slo_micros=100000)

20-03-13:06:05:44 INFO     [clipper_admin.py:236] [default-cluster] Application tf-app2 was successfully registered


In [22]:
deploy_tensorflow_model(
    clipper_conn,
    name="tf-mod2",
    version=1,
    input_type="doubles",
    func=predict,
    tf_sess_or_saved_model_path="/tmp/tf2/") # path to saved model files is used,

20-03-13:06:05:49 INFO     [deployer_utils.py:41] Saving function to /tmp/tmpp1fji782clipper
20-03-13:06:05:49 INFO     [deployer_utils.py:51] Serialized and supplied predict function
20-03-13:06:05:49 INFO     [tensorflow.py:264] TensorFlow model copied to: tfmodel 
20-03-13:06:05:49 INFO     [tensorflow.py:277] Using Python 3.6 base image
20-03-13:06:05:49 INFO     [clipper_admin.py:534] [default-cluster] Building model Docker image with model data from /tmp/tmpp1fji782clipper
20-03-13:06:05:50 INFO     [clipper_admin.py:539] [default-cluster] Step 1/2 : FROM clipper/tf36-container:0.4.1
20-03-13:06:05:50 INFO     [clipper_admin.py:539] [default-cluster]  ---> 3db42af800ff
20-03-13:06:05:50 INFO     [clipper_admin.py:539] [default-cluster] Step 2/2 : COPY /tmp/tmpp1fji782clipper /model/
20-03-13:06:05:50 INFO     [clipper_admin.py:539] [default-cluster]  ---> 28f56663858f
20-03-13:06:05:50 INFO     [clipper_admin.py:539] [default-cluster] Successfully built 28f56663858f
20-03-13:06:0

In [23]:
clipper_conn.link_model_to_app(
    app_name="tf-app2",
    model_name="tf-mod2")

20-03-13:06:05:54 INFO     [clipper_admin.py:303] [default-cluster] Model tf-mod2 is now linked to application tf-app2


In [24]:
query_address = clipper_conn.get_query_addr()
print(query_address)

localhost:1337


In [25]:
import requests, json, numpy as np
headers = {"Content-type": "application/json"}
requests.post("http://"+query_address+"/tf-app2/predict", headers=headers, data=json.dumps({
    "input": [.8,1.2]})).json()

{'query_id': 0,
 'output': -1.0,
 'default': True,
 'default_explanation': 'Failed to retrieve a prediction response within the specified latency SLO'}

In [39]:
clipper_conn.stop_all()

20-03-13:06:45:45 INFO     [clipper_admin.py:1424] [default-cluster] Stopped all Clipper cluster and all model containers
